In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import pandas as pd
import mmh3
import re

MAX_LEN_CHARS = 256*3

In [3]:
from datasets import load_metric

In [4]:
import nemo
import nemo.collections.nlp as nemo_nlp

model_distil = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name="punctuation_en_distilbert")

[NeMo W 2022-04-23 21:03:53 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-04-23 21:03:53 __init__:22] `pynini` is not installed ! 
    Please run the `nemo_text_processing/setup.sh` scriptprior to usage of this toolkit.
[NeMo W 2022-04-23 21:03:53 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


[NeMo I 2022-04-23 21:03:54 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.7.2/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo.
[NeMo I 2022-04-23 21:03:54 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.7.2/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo
[NeMo I 2022-04-23 21:03:54 common:704] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-23 21:03:56 tokenizer_utils:125] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmplzpcj62l/tokenizer.vocab_file, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-04-23 21:04:01 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    label_info_save_dir: null
    text_file: text_train.txt
    labels_file: labels_train.txt
    tokens_in_batch: null
    max_seq_length: 128
    num_samples: -1
    use_cache: true
    cache_dir: null
    get_label_frequences: false
    verbose: true
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    shuffle: true
    drop_last: false
    pin_memory: true
    num_workers: 8
    persistent_workers: true
    ds_item: punct_dataset_complete
    
[NeMo W 2022-04-23 21:04:01 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid co

[NeMo I 2022-04-23 21:04:05 save_restore_connector:157] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.7.2/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo.


In [5]:
df_t = pd.read_csv("data/eng_sentences.tsv", sep="\t", names=["id", "lang", "text"])
df_t = df_t[df_t["text"].str.len() < MAX_LEN_CHARS]

# shuffle
df_t["id"] = df_t["text"].map(lambda x: mmh3.hash64(x.encode('utf8'))[0])
df_t["text"] = df_t["text"].map(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))
df_t["text"] = df_t["text"].map(lambda x: re.sub(' +', ' ', x).strip())
df_t = df_t.sort_values("id")
len(df_t)

1582094

In [6]:
df_o = pd.read_csv("data/oss.tsv", sep="\t", names=["text"])
df_o = df_o[df_o["text"].str.len() < MAX_LEN_CHARS]

# shuffle
df_o["id"] = df_o["text"].map(lambda x: mmh3.hash64(x.encode('utf8'))[0])
df_o["text"] = df_o["text"].map(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))
df_o["text"] = df_o["text"].map(lambda x: re.sub(' +', ' ', x).strip())
df_o = df_o.sort_values("id")
len(df_o)

66939

In [7]:
df_g = pd.read_csv("data/gutenberg.tsv", sep="\t", names=["text"])
df_g = df_g[df_g["text"].str.len() < MAX_LEN_CHARS]

# shuffle
df_g["id"] = df_g["text"].map(lambda x: mmh3.hash64(x.encode('utf8'))[0])
df_g["text"] = df_g["text"].map(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))
df_g["text"] = df_g["text"].map(lambda x: re.sub(' +', ' ', x).strip())
df_g = df_g.sort_values("id")
len(df_g)

4102516

In [8]:
test_size = 10000
texts_t = list(df_t["text"])[-test_size:]
texts_t = [text for text in texts_t if text.isascii()]
len(texts_t)

10000

In [9]:
test_size = 10000
texts_o = list(df_o["text"])[-test_size:]
texts_o = [text for text in texts_o if text.isascii()]
len(texts_o)

10000

In [10]:
test_size = 10000
texts_g = list(df_g["text"])[-test_size:]
texts_g = [text for text in texts_g if text.isascii()]
len(texts_g)

10000

In [11]:
texts_t[1]

"Alice didn't see the dog."

In [12]:
texts_o[1]

"Since the JavaScript support doesn't understand ES7 constructs, features like IntelliSense might not be fully accurate."

In [13]:
texts_g[1]

'"I don\'t fear him, anyway he comes," replied Will Banion. "I don\'t like it, but all of this was forced on me."'

In [14]:
texts = texts_t + texts_o + texts_g

In [15]:
model_distil.add_punctuation_capitalization([
    texts_o[1],
    texts_o[1].replace(".", ""),
    texts_o[1].replace(",", ""),
    texts_o[1].replace(",", "").replace(".", ""),
], return_labels=True)

[NeMo I 2022-04-23 21:05:21 punctuation_capitalization_model:1068] Using batch size 4 for inference
[NeMo I 2022-04-23 21:05:21 punctuation_capitalization_infer_dataset:91] Max length: 27
[NeMo I 2022-04-23 21:05:21 data_preprocessing:358] Some stats of the lengths of the sequences:
[NeMo I 2022-04-23 21:05:21 data_preprocessing:360] Min: 23 |                  Max: 25 |                  Mean: 24.0 |                  Median: 24.0
[NeMo I 2022-04-23 21:05:21 data_preprocessing:366] 75 percentile: 24.25
[NeMo I 2022-04-23 21:05:21 data_preprocessing:367] 99 percentile: 24.97


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.31batch/s]


['OU OO OU OO OO OO OU OO OO OO OU OO OO OO OO OO',
 'OU OO OU OO OO OO OU OO OO OO OU OO OO OO OO .O',
 'OU OO OU OO OO OO OU OO OO OO OU OO OO OO OO OO',
 'OU OO OU OO OO OO OU ,O OO OO OU OO OO OO OO .O']

In [16]:
metric = load_metric("seqeval")

In [17]:
def clean_punctuation(s):
    return s.replace(",", "").replace(".", "").replace("!", "").replace("?", "")

In [18]:
texts_without_punctuation = [
    clean_punctuation(text) for text in texts
]
preds = model_distil.add_punctuation_capitalization(texts_without_punctuation, batch_size=128, return_labels=True)

[NeMo I 2022-04-23 21:05:56 punctuation_capitalization_infer_dataset:91] Max length: 64
[NeMo I 2022-04-23 21:05:56 data_preprocessing:358] Some stats of the lengths of the sequences:
[NeMo I 2022-04-23 21:05:56 data_preprocessing:360] Min: 0 |                  Max: 216 |                  Mean: 21.8305 |                  Median: 14.0
[NeMo I 2022-04-23 21:05:56 data_preprocessing:366] 75 percentile: 27.00
[NeMo I 2022-04-23 21:05:56 data_preprocessing:367] 99 percentile: 122.00


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 295/295 [02:59<00:00,  1.64batch/s]


In [19]:
true_predictions = [
    ["B-COMMA" if "," in token_pred else "O" for token_pred in pred.split(" ")]
    for pred in preds
]

In [20]:
true_labels = [
    ["B-COMMA" if "," in token else "O" for token in text.split(" ") if token == "" or clean_punctuation(token)]
    for text in texts
]

In [21]:
results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'COMMA': {'precision': 0.627561556791104,
  'recall': 0.49219747702850025,
  'f1': 0.5516976520904251,
  'number': 32105},
 'overall_precision': 0.627561556791104,
 'overall_recall': 0.49219747702850025,
 'overall_f1': 0.5516976520904251,
 'overall_accuracy': 0.9527329349164484}

In [22]:
# tatoeba
results = metric.compute(predictions=true_predictions[0:10000], references=true_labels[0:10000])
results

{'COMMA': {'precision': 0.7959770114942529,
  'recall': 0.6890547263681592,
  'f1': 0.7386666666666667,
  'number': 1206},
 'overall_precision': 0.7959770114942529,
 'overall_recall': 0.6890547263681592,
 'overall_f1': 0.7386666666666667,
 'overall_accuracy': 0.992344846441265}

In [23]:
# oss
results = metric.compute(predictions=true_predictions[10000:20000], references=true_labels[10000:20000])
results

{'COMMA': {'precision': 0.6790646289817713,
  'recall': 0.5947427834220287,
  'f1': 0.6341127922971114,
  'number': 6201},
 'overall_precision': 0.6790646289817713,
 'overall_recall': 0.5947427834220287,
 'overall_f1': 0.6341127922971114,
 'overall_accuracy': 0.9726602899705148}

In [24]:
# gutenberg
results = metric.compute(predictions=true_predictions[20000:30000], references=true_labels[20000:30000])
results

{'COMMA': {'precision': 0.6032076984763433,
  'recall': 0.4568386104137987,
  'f1': 0.5199179780199525,
  'number': 24698},
 'overall_precision': 0.6032076984763433,
 'overall_recall': 0.4568386104137987,
 'overall_f1': 0.5199179780199525,
 'overall_accuracy': 0.9329644345070536}

In [25]:
model_bert = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name="punctuation_en_bert")

[NeMo I 2022-04-23 21:09:22 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.7.2/punctuation_en_bert/93b0369b5e0d147f61895feffcbcfb88/punctuation_en_bert.nemo.
[NeMo I 2022-04-23 21:09:22 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.7.2/punctuation_en_bert/93b0369b5e0d147f61895feffcbcfb88/punctuation_en_bert.nemo
[NeMo I 2022-04-23 21:09:22 common:704] Instantiating model from pre-trained checkpoint
[NeMo I 2022-04-23 21:09:26 tokenizer_utils:125] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpst0h_odi/tokenizer.vocab_file, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-04-23 21:09:31 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    label_info_save_dir: null
    text_file: text_train.txt
    labels_file: labels_train.txt
    tokens_in_batch: null
    max_seq_length: 128
    num_samples: -1
    use_cache: true
    cache_dir: null
    get_label_frequences: false
    verbose: true
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    shuffle: true
    drop_last: false
    pin_memory: true
    num_workers: 8
    persistent_workers: true
    ds_item: punct_dataset_complete
    
[NeMo W 2022-04-23 21:09:31 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid co

[NeMo I 2022-04-23 21:09:36 save_restore_connector:157] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.7.2/punctuation_en_bert/93b0369b5e0d147f61895feffcbcfb88/punctuation_en_bert.nemo.


In [26]:
preds = model_bert.add_punctuation_capitalization(texts_without_punctuation, batch_size=128, return_labels=True)

[NeMo I 2022-04-23 21:10:09 punctuation_capitalization_infer_dataset:91] Max length: 64
[NeMo I 2022-04-23 21:10:09 data_preprocessing:358] Some stats of the lengths of the sequences:
[NeMo I 2022-04-23 21:10:09 data_preprocessing:360] Min: 0 |                  Max: 216 |                  Mean: 21.8305 |                  Median: 14.0
[NeMo I 2022-04-23 21:10:09 data_preprocessing:366] 75 percentile: 27.00
[NeMo I 2022-04-23 21:10:09 data_preprocessing:367] 99 percentile: 122.00


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 295/295 [06:16<00:00,  1.27s/batch]


In [27]:
true_predictions = [
    ["B-COMMA" if "," in token_pred else "O" for token_pred in pred.split(" ")]
    for pred in preds
]

In [28]:
results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'COMMA': {'precision': 0.6923850126512079,
  'recall': 0.6222083787572029,
  'f1': 0.6554235842246866,
  'number': 32105},
 'overall_precision': 0.6923850126512079,
 'overall_recall': 0.6222083787572029,
 'overall_f1': 0.6554235842246866,
 'overall_accuracy': 0.9613411691517108}

In [29]:
# tatoeba
results = metric.compute(predictions=true_predictions[0:10000], references=true_labels[0:10000])
results

{'COMMA': {'precision': 0.8283712784588442,
  'recall': 0.7844112769485904,
  'f1': 0.8057921635434413,
  'number': 1206},
 'overall_precision': 0.8283712784588442,
 'overall_recall': 0.7844112769485904,
 'overall_f1': 0.8057921635434413,
 'overall_accuracy': 0.9940633503013891}

In [30]:
# oss
results = metric.compute(predictions=true_predictions[10000:20000], references=true_labels[10000:20000])
results

{'COMMA': {'precision': 0.7265179034769071,
  'recall': 0.6773101112723754,
  'f1': 0.7010515773660491,
  'number': 6201},
 'overall_precision': 0.7265179034769071,
 'overall_recall': 0.6773101112723754,
 'overall_f1': 0.7010515773660491,
 'overall_accuracy': 0.9769899338990563}

In [31]:
# gutenberg
results = metric.compute(predictions=true_predictions[20000:30000], references=true_labels[20000:30000])
results

{'COMMA': {'precision': 0.6763042685151405,
  'recall': 0.6004534780144142,
  'f1': 0.6361257667395874,
  'number': 24698},
 'overall_precision': 0.6763042685151405,
 'overall_recall': 0.6004534780144142,
 'overall_f1': 0.6361257667395874,
 'overall_accuracy': 0.9454179870349221}